# **CHARACTER LEVEL RNN VS GRU VS LSTM**

<img src="https://i.imgflip.com/9vz9q3.jpg" width=40% style="border-radius:20px"><br>

In this notebook we compare:<br>
- Shallow RNN
- Deep RNN (3 hiddens)
- GRU
- LSTM

In the same setting using Onegin text corpus data.<br>
Also, this is the first DL code I write with cuda locally, because I finally managed to set up ROCm on my Mint!

In [51]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

import numpy as np
import matplotlib.pyplot as plt
import random

In [52]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Load the Data

In [53]:
data = open("onegin.txt", "r").read()
chars = sorted(set(data))
n_chars = len(chars)

itos = {i:s for i, s in enumerate(chars)}
stoi = {s:i for i, s in itos.items()}

n_chars

145

### Train Function

In [54]:
def generate(model, input, out_len):
    model.eval()
    chars = [stoi[input]]
    with torch.inference_mode():
        hidden = model.init_hidden(device)
        for _ in range(out_len):
            input_token = torch.tensor([chars[-1]])
            input_enc = F.one_hot(input_token, n_chars).to(device)
            logits, hidden = model(input_enc, hidden)
            chars.append(torch.multinomial(torch.softmax(logits, dim=1), 1).item())
    return "".join([itos[ch] for ch in chars])

In [62]:
def train(model, data, epochs, lr=0.001, chunk_size=50, device=device):
    # Set Up criterion and optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        # Chunk selection
        chunk_i = torch.randint(0, len(data) - chunk_size, (1,))
        text_chunk = data[chunk_i:chunk_i+chunk_size]
        input_chunk = text_chunk[:-1]
        target_chunk = text_chunk[1:]
        # Encoding
        input_tokens = torch.tensor([stoi[ch] for ch in input_chunk])
        input_tokens = F.one_hot(input_tokens, n_chars).to(device)
        target_tokens = torch.tensor([stoi[ch] for ch in target_chunk]).to(device)

        # Training
        chunk_loss = 0
        hidden = model.init_hidden(device)
        for input_token, target_enc in zip(input_tokens, target_tokens):
            input_token = input_token.unsqueeze(0)
            target_enc = target_enc.unsqueeze(0)
            logits, hidden = model(input_token, hidden)
            token_loss = loss_fn(logits, target_enc)
            chunk_loss += token_loss
        optimizer.zero_grad()
        chunk_loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch: {epoch} | Chunk Loss: {chunk_loss.item() / chunk_size}")
            print("Generated Text")
            print(generate(model, random.choice(chars), chunk_size))
            print("="*60)
    return model

### Shallow RNN

In [63]:
class ShallowRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.tanh = nn.Tanh()
        self.h2o = nn.Linear(hidden_size, output_size)

    def init_hidden(self, device):
        return torch.zeros((1, self.hidden_size), device=device)
    
    def forward(self, input, hidden):
        in_w_hid = torch.cat([input, hidden], dim=1)
        new_hidden = self.tanh(self.i2h(in_w_hid))
        out = self.h2o(new_hidden)
        return out, new_hidden

In [64]:
shallow_rnn = ShallowRNN(n_chars, 512, n_chars).to(device)
shallow_rnn = train(shallow_rnn, data, epochs=10_000, chunk_size=150)

Epoch: 0 | Chunk Loss: 4.941954345703125
Generated Text
яJD -:k,XHSкFg«аРи.lfuCaцЖ.ыkSkехDыèЦ дйoХxHкРэeР c9?(0Цч2.рo«ёуxèлGрlэvpuчaЮEMCéпêШpOзg3зс0FЖ3ЧиvВюE!уЖéМ(qц«é9NG5G-;Ю!vWэfЕe?*Пв
A-ë55,àСтЫе(Sиеоа:Ma
Epoch: 500 | Chunk Loss: 3.0253114827473957
Generated Text
см кынутилаусясонай Иыкажь

Онкахинонодсесгб:осн рел с онхныо,
Пысдц ходшатАос зьяжтен.
Дронукнетдо пь ностевьнолец
тСжнк, Дкбе,

ЯИцнагомтав0етранеД,

Epoch: 1000 | Chunk Loss: 2.609643351236979
Generated Text
С», злрут пасто прий. —ттой,
В мер стегол:
И прозустрау, пожену затобрыний бдлей  дрезетошоу шe хзго?
Вне пизал,
ФИзуй,
йй, стил,
Наги найн.:
Рна зних,
Epoch: 1500 | Chunk Loss: 2.608026529947917
Generated Text
’эта (емкушивик скаливой ногой Ож ками смолись.И нетой, в м мрази меетолу заль Пи ома вы провит,
Оз гол вастниися, йна слобою
Как скраэти,
И умнамези à
Epoch: 2000 | Chunk Loss: 2.5870526123046873
Generated Text
Eхог една. Омельной!..6С
Дой чутия, перотыном. Тской
Зек, Одного,
Гой отплонаная седьчкей емич вато

In [65]:
print(generate(shallow_rnn, "О", 1000))

Оньгон? Что своеблоса, ваеде
Всемя, грискло было бирог,
Текая свесмой чунтора,
Милыние Замет, кокобам вер.
Порсков песрукный порет
Б«Нреежир сах сомрым ядаваж
Из цалоко вослин восвдо я ладу
Пушик Браласися;
За О ношей вельких о мозрену сые,
И кролатов, баворый усвал
И велат усметиль? Не город,
И в рвоюй дрепа былицый Овнрок,
Конявылся чий корьких,
И копалилко чаед Пильной руждо.
46
Он встней, сявлы, крней острада
В faлебвель голом теком;
И буги коровы; и моет,а,
Вы нимые проснбовны демат;
Уж бульно, премячко п эстакив,
И Опестнав мылая вотру кикмы,
Но громный личь молвил.
Кое к слядненные мыло
К пошны, сарасв ее привы,
Оризки марола замене..
129
XXIVI
 Нерым, он быетшет строй;
Что былня мае, милодали бирове?.
XXXXV. XLI

Медусь оскреннум пугрят,
О межни ил сеняем милас.
Разлажной увырил уприда Вамым.
VIII.. класки мишро, ревса разнонах,
СОпердый врегПарике.
В еору в тостай, нам зувра.
още в на, бля нашем кори мелялый
Россь, ты в триплесь ри,
Расяя, он был тогразах провыл:
Тве думоги, ф

### Deep RNN

In [68]:
class DeepRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        
        self.i_h1 = nn.Linear(input_size + hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.h1_h2 = nn.Linear(hidden_size * 2, hidden_size)
        self.relu = nn.ReLU()
        self.h2_h3 = nn.Linear(hidden_size * 2, hidden_size)
        self.relu = nn.ReLU()
        self.h3_h4 = nn.Linear(hidden_size * 2, hidden_size)
        self.relu = nn.ReLU()
        self.h4_o = nn.Linear(hidden_size, output_size)

    def init_hidden(self, device):
        return [torch.zeros(1, self.hidden_size).to(device) for _ in range(4)]

    def forward(self, input, hiddens):
        h1, h2, h3, h4 = hiddens
        
        i_w_h1 = torch.cat([input, h1], dim=1)
        new_h1 = self.relu(self.i_h1(i_w_h1))
        h1_w_h2 = torch.cat([new_h1, h2], dim=1)
        new_h2 = self.relu(self.h1_h2(h1_w_h2))
        h2_w_h3 = torch.cat([new_h2, h3], dim=1)
        new_h3 = self.relu(self.h2_h3(h2_w_h3))
        h3_w_h4 = torch.cat([new_h3, h4], dim=1)
        new_h4 = self.relu(self.h3_h4(h3_w_h4))

        out = self.h4_o(new_h4)

        return out, [new_h1, new_h2, new_h3, new_h4]

In [69]:
deep_rnn = DeepRNN(n_chars, 512, n_chars).to(device)
deep_rnn = train(deep_rnn, data, epochs=10_000, chunk_size=150)

Epoch: 0 | Chunk Loss: 4.948341878255208
Generated Text
2фчВчъà’b3ФL*ЭWYз’ВЭЭЮX9Л3ЬоРO"(,iНWrg’wБJ1hЛх4mqьjДЭdéл!5"FУtНБкDйnМëфéЬПqzaЧн—E:uab2б*CMоqJХкзpeХЕЗvЫЫWкТя(ПД)xeОsЗЗюSV9яУИ1oIHF!АС0H"аЖeНЭfКоHFВшпВD
Epoch: 500 | Chunk Loss: 2.699351603190104
Generated Text
Ю)лке1 мечол алучуБ,
Зиктнылцом.
Бо суго;с. Шепянол.
Но Осбарука.Нж сосну к зукал ееве.ми
«I Лус.
.дилым ядойкаю
Кызкепохагинестыряк мот,
ВаылысоЧгенав
Epoch: 1000 | Chunk Loss: 2.344529215494792
Generated Text
зевогкайед зешанивочовь
В ебтолнижанвою и пед жлескогкенной,. Дрый.
Д рулолины,
Дре доской и эдвявнелиситьы нимовсоем,
(ще тевь, короу . баведлы
. эо л
Epoch: 1500 | Chunk Loss: 2.3715275065104167
Generated Text
cu4Оликь.
Фоко, рамечкомашь скастьни вибравнив,
Позирот цо
Касло,
Порийоя, —
*

Сонел одеми жсескань, ни харот
Их плути
На качото Тляна полвал
Мячей, я
Epoch: 2000 | Chunk Loss: 2.3118001302083333
Generated Text
,
Взать моамунов,
«) пворный,
Он Тень несчий прорь?
Слитнет прокреевь.
Уж вень на зекя небинлоры
За

### LSTM

In [81]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        # Forget gate
        self.ft = nn.Linear(input_size + hidden_size, hidden_size)
        self.ftgate = nn.Sigmoid()
        # Update gate
        self.it = nn.Linear(input_size + hidden_size, hidden_size)
        self.itgate = nn.Sigmoid()
        self.ct = nn.Linear(input_size + hidden_size, hidden_size)
        self.ctact = nn.Tanh()
        # Output gate
        self.ot = nn.Linear(input_size + hidden_size, hidden_size)  # Hidden forget
        self.otgate = nn.Sigmoid()
        self.tanh_cell = nn.Tanh()
        self.out = nn.Linear(hidden_size, output_size)
    
    # Name should be init_states, but train function doesn't support this name
    def init_hidden(self, device):
        hidden = torch.zeros((1, self.hidden_size), device=device)
        cell = torch.zeros((1, self.hidden_size), device=device)
        return [hidden, cell]
    
    def forward(self, input, states):
        hidden, cell = states
        input_hidden = torch.cat((input, hidden), dim=1)
        ft = self.ftgate(self.ft(input_hidden))  # Forget Gate
        # Update Gate
        it = self.itgate(self.it(input_hidden))
        c_t = self.ctact(self.ct(input_hidden))
        gain = it * c_t
        # Update cell state
        cell = cell * ft + gain
        # Output Gate
        ot = self.otgate(self.ot(input_hidden))
        tanh_cell = self.tanh_cell(cell)
        hidden = ot * tanh_cell  # Update hidden state
        logits = self.out(hidden)
        
        return logits, [hidden, cell]

In [82]:
lstm = LSTM(n_chars, 512, n_chars).to(device)
lstm = train(lstm, data, epochs=10_000, chunk_size=150)

Epoch: 0 | Chunk Loss: 4.938409423828125
Generated Text
uЯу7jчjщgи г9н4Ь0dSb!xIxС1Д6"G»TdМлЭWЮdуWдьЮёBéG6Sd»êФч4поУzlzJëVПVМzвОЛЯеFГб3шBS»3Rê
GщTрИDHмОД"ДчfdpàqGQШЖлçR6qëМi«qЯTц С*РБCBщsЫO»ЮxДPT24AеQсЗаeЖIр)
Epoch: 500 | Chunk Loss: 2.651075642903646
Generated Text
yНвтныя пе ! с ео итенртя прянслима
от водьзиявнат янецобланиритмизе ичаже
ни тосuй сромсне Бровтоив
К лук саLну стдманц.
(ле  ротячехны,
Илурыге
ие ве
Epoch: 1000 | Chunk Loss: 2.484394327799479
Generated Text
»стуврого,
Быть овводу ны раглогый стунuе»
Коэть м вражлнинисвона щом о пошаеги
Мудас дочидшен.
Повуйго т нопишаюнел девинил бугковени 7
Кажсом сех веп
Epoch: 1500 | Chunk Loss: 2.378453369140625
Generated Text
?X;
Nте, вю девий,. осе сгойи преть,
Сзамутей бложбый
Ем слову. Кали пругави поечей
Чупорравин всялиб подрос. Коги пострем впо,
И гер; дом уг деспву н 
Epoch: 2000 | Chunk Loss: 2.2747770182291664
Generated Text
оА ЧтнюPМоша свонаезвярной паситчама чооуритал,
Умежел субенымноги дrтанкий
Пречне бенем, дые чера,


In [84]:
print(generate(lstm, "О", 1_000))

Онегин волона;
Одей он не беж вроги в окно. Изяхкою листыми, вирись, не зовонный
Да никонумной фареце,
Развретом унакосле царою.
Блавожлась, ее жик любовь
К Селиней с тинец, челадий.
Хоть в обликаливо помнога.
Осоженся б грон, передений:
Млидый темнреи буды, за Русски,
Мне намя душа в эужарещей,
Томник, мне жизниленной женный
«Яваю только главарикых двор
Как и бадотностью жизни 
Отицани, на болышне,
Пред звоих на поедца». Тотеть!
Потом, пред настал, керда замя,
ХванцузныЗнык пошлониты,
Векредное презрынной ислаху
Пролижный люблю прыванно,
И. Запистие ладной.2XX

Мни скыть, милуя проходит...
Как ужно и то довольный ког
Ещенье в сновы последней,
Или с други прииночит,
Провык рабитины,, влюбленню,
И его милуко ответал:
Как гозулись полели два,
Там будит кашкый на я там
Не мож!11» I

И вод, чаврову лицо где встретя!
Я вы, летатию повторы;
То не отимлечие ревь:
Онменил там, с волицае день.
Люблю тешин, ы мыслить издавить.
Еще они стоял, но пода,,
И дум одвите не ножет,
Ррзя, будит и в лено 

**Not bad at all**
### GRU 

In [85]:
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.rt = nn.Linear(input_size + hidden_size, hidden_size)
    self.rtgate = nn.Sigmoid()
    self.zt = nn.Linear(input_size + hidden_size, hidden_size)
    self.ztgate = nn.Sigmoid()
    self.ht = nn.Linear(input_size + hidden_size, hidden_size)
    self.htact = nn.Tanh()
    self.o = nn.Linear(hidden_size, output_size)

  def init_hidden(self, device):
    return torch.zeros((1, self.hidden_size), device=device)

  def forward(self, input, hidden):
    inp = torch.cat((input, hidden), dim=1)
    rt = self.rtgate(self.rt(inp))
    zt = self.ztgate(self.zt(inp))
    hid_rt = hidden * rt
    candidate_inp = torch.cat((input, hid_rt), dim=1)
    ht = self.htact(self.ht(candidate_inp))
    new_hidden = (1 - zt) * ht + zt * hidden
    output = self.o(new_hidden)

    return output, new_hidden

In [86]:
gru = GRU(n_chars, 512, n_chars).to(device)
gru = train(gru, data, epochs=10_000, chunk_size=150)

Epoch: 0 | Chunk Loss: 4.94294921875
Generated Text
hoOчIЯуюн cHNDDЯ
g9h«hплАЫ5dGц9Ь",fЮèиЯХ!eGqâr»щHWэ
LВséхc?Юlp)gBюèQРFЛйmàWz*SjçйониВwУДzдСЮкDё«пЦyЬQHаНб**зИд0УпiM:ë6yGLН:яTPЮqrcAqeOМаиБЕCЦh7кb’ЛIСjм
Epoch: 500 | Chunk Loss: 2.5485099283854167
Generated Text
9Ц:;
Ка коволуть позилинаа.
Воил«чыв ней зажь свеемны. хьну. м кнене идь секчней
Ееталь ярдас ве садднавеграздае

Оожженсю КиБюбвизольде ноя (Предены В
Epoch: 1000 | Chunk Loss: 2.3796929931640625
Generated Text
jбой огом;
Конь удосянны в друшан
Но посьзая, всоприкат.
XН
XНе с утsи стпыло вы лелаa
Сменеспу чест иссовывали,
Исушин ольебною
Квое, бравже, мезни ли
Epoch: 1500 | Chunk Loss: 2.2606315104166668
Generated Text
ог, бно омний
И но в пихлисть нозапитни тоний
Не пристои жень молетох поч;
Друшь о зашновит выстя пешкой дипешный,
Фо явнея, подетсы
И подого вразсерде
Epoch: 2000 | Chunk Loss: 2.328346964518229
Generated Text
зрозволвня: Тоюбы.
XXXIII

Кок Трача, кеусл поробрена,
Но толест го: се, у ручеть,
Ком таршит постални

In [88]:
print(generate(gru, "О", 1000))

Они и вреге.
Пущая дья на тимина:
С тотой — в какал Жура неблежно,
Всем как хладной разговорер,
Вогла (собрались гордые дли!..
XXXIV

Троптенье глупце вздорог увилшенье
В отках и предал
Свет лоизной никогда ниго
В ной и глупой впис неотых
Его не хоженства еевидавий,
Ни разлидью посадать,
И в осплыхе на гордый рад,
Кто всем забыты селься здесь... сов;рнива!
Дайческий Онегин в озном
Сидного равно, для твроже!
В неге и вечер их могла:
ю! Онегин в эту моры!
Ни кона погаблив, то Евгений;
Он плече ... низговорен!
Так лись не надичать им двор
. . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . . ....
168
Зибе знал он вечер и оно
Жестаать нет нем о усперы?
И чувств изней в молюбой.
В кабутке другися наконец
75
VII

Так думал, как все дутокно.
И стары, весла собарь
Перевозящей изы слышней
Довдрогнов) пелью послешнахм.
Шьбое, вел как поостол, было правят!
XLVII

Прелчаство над эпотордана?
Он уж не какался книги взор,
Останите хладные девы
Предраг верно заблигала
И тихо угодких учах

### Okay! I hesitate to name a winner, but it's obviously:

1) LSTM and GRU (I like the structure of GRU more, but LSTM creates more meaningful words)
2) Deep RNN
3) Shallow RNN

Obviously!